In [1]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ArgParse",
"Base64",
"BioSequences",
"DataFrames",
"Dates",
"DelimitedFiles",
"FASTX",
"GLM",
"HTTP",
"JSON",
"Graphs",
"MetaGraphs",
"MD5",
"Statistics",
"StatsPlots",
"uCSV",
"CodecZlib",
"YAML",
"Revise",
"Kmers",
"StatsBase",
"ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

  Activating environment at `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Project.toml`
  No Changes to `~/work/Mycelia/projects/viral-pangenome-discovery/notebooks/Manifest.toml`


In [2]:
data_dir = joinpath(dirname(pwd()), "data")

"/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data"

In [3]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> "trim_galore" in readdir(x), SRR_paths)

32-element Vector{String}:
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399460"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399461"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399462"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399463"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399464"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399465"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399466"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399467"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399468"
 "/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399469"
 "/home/jovyan/work/Mycelia/projects/viral-pangen

In [ ]:
ProgressMeter.@showprogress for SRR_path in SRR_paths
    SRR = basename(SRR_path)
    @show SRR

    out_dir = joinpath(SRR_path, "megahit")

    trimmed_forward_reads = joinpath(SRR_path, "trim_galore", "$(SRR)_1_val_1.fq.gz")
    trimmed_reverse_reads = joinpath(SRR_path, "trim_galore", "$(SRR)_2_val_2.fq.gz")

    # megahit: MEGAHIT v1.2.9

    # contact: Dinghua Li <voutcn@gmail.com>

    # Usage:
    #   megahit [options] {-1 <pe1> -2 <pe2> | --12 <pe12> | -r <se>} [-o <out_dir>]

    #   Input options that can be specified for multiple times (supporting plain text and gz/bz2 extensions)
    #     -1                       <pe1>          comma-separated list of fasta/q paired-end #1 files, paired with files in <pe2>
    #     -2                       <pe2>          comma-separated list of fasta/q paired-end #2 files, paired with files in <pe1>
    #     --12                     <pe12>         comma-separated list of interleaved fasta/q paired-end files
    #     -r/--read                <se>           comma-separated list of fasta/q single-end files

    # Optional Arguments:
    #   Basic assembly options:
    #     --min-count              <int>          minimum multiplicity for filtering (k_min+1)-mers [2]
    #     --k-list                 <int,int,..>   comma-separated list of kmer size
    #                                             all must be odd, in the range 15-255, increment <= 28)
    #                                             [21,29,39,59,79,99,119,141]

    #   Another way to set --k-list (overrides --k-list if one of them set):
    #     --k-min                  <int>          minimum kmer size (<= 255), must be odd number [21]
    #     --k-max                  <int>          maximum kmer size (<= 255), must be odd number [141]
    #     --k-step                 <int>          increment of kmer size of each iteration (<= 28), must be even number [12]

    #   Advanced assembly options:
    #     --no-mercy                              do not add mercy kmers
    #     --bubble-level           <int>          intensity of bubble merging (0-2), 0 to disable [2]
    #     --merge-level            <l,s>          merge complex bubbles of length <= l*kmer_size and similarity >= s [20,0.95]
    #     --prune-level            <int>          strength of low depth pruning (0-3) [2]
    #     --prune-depth            <int>          remove unitigs with avg kmer depth less than this value [2]
    #     --disconnect-ratio       <float>        disconnect unitigs if its depth is less than this ratio times 
    #                                             the total depth of itself and its siblings [0.1]  
    #     --low-local-ratio        <float>        remove unitigs if its depth is less than this ratio times
    #                                             the average depth of the neighborhoods [0.2]
    #     --max-tip-len            <int>          remove tips less than this value [2*k]
    #     --cleaning-rounds        <int>          number of rounds for graph cleanning [5]
    #     --no-local                              disable local assembly
    #     --kmin-1pass                            use 1pass mode to build SdBG of k_min

    #   Presets parameters:
    #     --presets                <str>          override a group of parameters; possible values:
    #                                             meta-sensitive: '--min-count 1 --k-list 21,29,39,49,...,129,141'
    #                                             meta-large: '--k-min 27 --k-max 127 --k-step 10'
    #                                             (large & complex metagenomes, like soil)

    #   Hardware options:
    #     -t/--num-cpu-threads     <int>          number of CPU threads [# of logical processors]

    #   Output options:
    #     -o/--out-dir             <string>       output directory [./megahit_out]
    #     --out-prefix             <string>       output prefix (the contig file will be OUT_DIR/OUT_PREFIX.contigs.fa)
    #     --min-contig-len         <int>          minimum length of contigs to output [200]

    # median_read_length = Statistics.median(vcat(Mycelia.determine_read_lengths(trimmed_forward_reads), Mycelia.determine_read_lengths(trimmed_reverse_reads)))

    # k_lengths = [i for i in [21,29,39,59,79,99,119,141] if i < joint_median_read_length]
    # k_lengths_string = join(string.(k_lengths), ',')

    # 2023-02-19 00:51:46 - ALL DONE. Time elapsed: 2031.380950 seconds 

    initial_assembled_fasta = "$(out_dir)/final.contigs.fa"
    assembled_fastg = replace(initial_assembled_fasta, ".fa" => ".fastg")

    if !isfile(initial_assembled_fasta)
        run(`megahit --num-cpu-threads $(Sys.CPU_THREADS) -1 $trimmed_forward_reads -2 $trimmed_reverse_reads -o $(out_dir)`)
        # run(`conda run --live-stream --no-capture-output -n viral-pangenome-discovery megahit --num-cpu-threads $(Sys.CPU_THREADS) -1 $trimmed_forward_reads -2 $trimmed_reverse_reads -o $(out_dir)`)
    end

    # read in the assembled fasta file and parse contig identifiers to get final k length
    final_k_lengths = unique([replace(first(split(FASTX.identifier(record), '_')), r"^k" => "") for record in Mycelia.open_fastx(initial_assembled_fasta)])
    @assert length(final_k_lengths) == 1
    final_k_length = parse(Int, first(final_k_lengths))
    if !isfile(assembled_fastg)
        run(pipeline(`megahit_toolkit contig2fastg $(final_k_length) $(initial_assembled_fasta)`, assembled_fastg))
    end

    assembled_gfa = "$(assembled_fastg).gfa"
    if !isfile(assembled_gfa)
        run(`Bandage reduce $(assembled_fastg) $(assembled_gfa)`)
    end

    assembled_fasta = assembled_gfa * ".fna"
    open(assembled_fasta, "w") do io
        fastx_io = FASTX.FASTA.Writer(io)
        gfa_graph = Mycelia.parse_gfa(assembled_gfa)
        for v in Graphs.vertices(gfa_graph)
            record = FASTX.FASTA.Record(gfa_graph.vprops[v][:identifier], gfa_graph.vprops[v][:sequence])
            write(fastx_io, record)
        end
        close(fastx_io)
    end

    # generate a bandage plot of the assembly graph
    bandage_outfile = "$(assembled_gfa).bandage.jpg"
    if !isfile(bandage_outfile)
        run(`Bandage image $(assembled_gfa) $bandage_outfile`)
    end

    # map reads to the assembly and run qualimap QC
    bwt_index = "$(assembled_fasta).bwt"
    if !isfile(bwt_index)
        run(`bwa index $(assembled_fasta)`)
    end

    # mapped_reads_bam = "$(assembled_fasta).bwa.sorted.marked_duplicates.bam"
    mapped_reads_bam = "$(assembled_fasta).bwa.bam"
    # sorting and deduping
    # if !isfile(mapped_reads_bam)
    #     run(pipeline(
    #         `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(Sys.CPU_THREADS) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
    #         `samtools collate --threads $(Sys.CPU_THREADS) -O - -`,
    #         `samtools fixmate --threads $(Sys.CPU_THREADS) -m - -`,
    #         `samtools sort --threads $(Sys.CPU_THREADS)`,
    #         `samtools markdup --threads $(Sys.CPU_THREADS) - -`,
    #         `samtools view --threads $(Sys.CPU_THREADS) -buh`,
    #         mapped_reads_bams))
    # end
    # sorting - 50Gb - 1594.823 sec
    if !isfile(mapped_reads_bam)
        run(pipeline(
            `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(Sys.CPU_THREADS) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
            `samtools sort --threads $(Sys.CPU_THREADS)`,
            `samtools view --threads $(Sys.CPU_THREADS) -buh`,
            mapped_reads_bam))
    end
    # no sorting - 50Gb - 1762.476216 seconds
    # if !isfile(mapped_reads_bam)
    #     @time run(pipeline(
    #         `bwa mem -R "@RG\tID:$(SRR)\tSM:bar" -t $(Sys.CPU_THREADS) $(assembled_fasta) $(trimmed_forward_reads) $(trimmed_reverse_reads)`,
    #         `samtools view --threads $(Sys.CPU_THREADS) -buh`,
    #         mapped_reads_bam))
    # end

    if !isfile("$(mapped_reads_bam).bai")
        run(`samtools index $(mapped_reads_bam)`)
    end

    qualimap_report_pdf = "$(out_dir)/qualimap/report.pdf"
    qualimap_report_txt = "$(out_dir)/qualimap/genome_results.txt"

    if !isfile(qualimap_report_pdf) || !isfile(qualimap_report_txt)
        run(`
            conda run --live-stream -n viral-pangenome-discovery
            qualimap bamqc
            -nt $(Sys.CPU_THREADS)
            -bam $(mapped_reads_bam)
            -outdir $(out_dir)/qualimap
            -outformat PDF:HTML
            --output-genome-coverage $(mapped_reads_bam).genome_coverage.txt
            --java-mem-size=4G
            `)
    end
end

# qualimap_contig_coverage_table = Mycelia.parse_qualimap_contig_coverage(qualimap_report_txt)
# qualimap_contig_coverage_table[!, "% Mapped bases"] = round.(qualimap_contig_coverage_table[!, "Mapped bases"] ./ sum(qualimap_contig_coverage_table[!, "Mapped bases"]) .* 100, digits=3);

SRR = "SRR6399459"
SRR = "SRR6399460"


Progress:   6%|██▌                                      |  ETA: 0:00:58

SRR = "SRR6399461"


2023-02-23 22:38:58 - MEGAHIT v1.2.9
2023-02-23 22:38:58 - Using megahit_core with POPCNT and BMI2 support
2023-02-23 22:38:58 - Convert reads to binary library
2023-02-23 22:44:22 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399461/trim_galore/SRR6399461_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399461/trim_galore/SRR6399461_2_val_2.fq.gz): pe, 136163688 reads, 151 max length'
2023-02-23 22:44:29 - b'INFO  utils/utils.h                 :  152 - Real: 330.9140\tuser: 147.9503\tsys: 10.6447\tmaxrss: 254416'
2023-02-23 22:44:29 - k-max reset to: 141 
2023-02-23 22:44:29 - Start assembly. Number of CPU threads 48 
2023-02-23 22:44:29 - k list: 21,29,39,59,79,99,119,141 
2023-02-23 22:44:29 - Memory used: 121546591027
2023-02-23 22:44:29 - Extract solid (k+1)-mers for k = 21 
2023-02-23 22:54:23 - Build graph for k = 21 
2023-02-23 22:56:14 - Assemble contigs f

Java memory size is set to 4G
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 37
Max memory (Mb): 4294
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 24232
Chunk of reads size: 1000
Number of threads: 48
Processed 2423 out of 24232 windows...
Processed 4846 out of 24232 windows...
Processed 7269 out of 24232 windows...
Processed 9692 out of 24232 windows...
Processed 12115 out of 24232 windows...
Processed 14538 out of 24232 windows...
Processed 16961 out of 24232 windows...
Processed 19384 out of 24232 windows...
Processed 21807 out of 24232 windows...
Processed 24230 out of 24232 windows...
Total processed windows:24232
Number of reads: 136163688
Number of valid reads: 139883903
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 132583918
Num mapped first of pair: 66447504
Num mapped second of pair: 6613641

...

2023-02-24 01:05:56 - MEGAHIT v1.2.9
2023-02-24 01:05:56 - Using megahit_core with POPCNT and BMI2 support
2023-02-24 01:05:56 - Convert reads to binary library
2023-02-24 01:10:17 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399462/trim_galore/SRR6399462_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399462/trim_galore/SRR6399462_2_val_2.fq.gz): pe, 124896854 reads, 151 max length'
2023-02-24 01:10:21 - b'INFO  utils/utils.h                 :  152 - Real: 264.8290\tuser: 140.5360\tsys: 12.6366\tmaxrss: 254076'
2023-02-24 01:10:21 - k-max reset to: 141 
2023-02-24 01:10:21 - Start assembly. Number of CPU threads 48 
2023-02-24 01:10:21 - k list: 21,29,39,59,79,99,119,141 
2023-02-24 01:10:21 - Memory used: 121546591027
2023-02-24 01:10:21 - Extract solid (k+1)-mers for k = 21 
2023-02-24 01:19:48 - Build graph for k = 21 
2023-02-24 01:20:49 - Assemble contigs f

Java memory size is set to 4G
Launching application...

QualiMap v.2.2.2-dev
Built on 2019-11-11 14:05

Selected tool: bamqc
Available memory (Mb): 37
Max memory (Mb): 4294
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 6483
Chunk of reads size: 1000
Number of threads: 48
Processed 648 out of 6483 windows...
Processed 1296 out of 6483 windows...
Processed 1944 out of 6483 windows...
Processed 2592 out of 6483 windows...
Processed 3240 out of 6483 windows...
Processed 3888 out of 6483 windows...
Processed 4536 out of 6483 windows...
Processed 5184 out of 6483 windows...
Processed 5832 out of 6483 windows...
Processed 6480 out of 6483 windows...
Total processed windows:6483
Number of reads: 124896854
Number of valid reads: 132348735
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 123797785
Num mapped first of pair: 62000508
Num mapped second of pair: 61797277
Num singletons: 7

Progress:  12%|█████▏                                   |  ETA: 1 days, 7:16:24

SRR = "SRR6399463"


2023-02-24 03:06:57 - MEGAHIT v1.2.9
2023-02-24 03:06:57 - Using megahit_core with POPCNT and BMI2 support
2023-02-24 03:06:57 - Convert reads to binary library
2023-02-24 03:10:59 - b'INFO  sequence/io/sequence_lib.cpp  :   75 - Lib 0 (/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399463/trim_galore/SRR6399463_1_val_1.fq.gz,/home/jovyan/work/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399463/trim_galore/SRR6399463_2_val_2.fq.gz): pe, 93012880 reads, 151 max length'
2023-02-24 03:11:04 - b'INFO  utils/utils.h                 :  152 - Real: 246.6784\tuser: 102.7976\tsys: 10.4561\tmaxrss: 254220'
2023-02-24 03:11:04 - k-max reset to: 141 
2023-02-24 03:11:04 - Start assembly. Number of CPU threads 48 
2023-02-24 03:11:04 - k list: 21,29,39,59,79,99,119,141 
2023-02-24 03:11:04 - Memory used: 121546591027
2023-02-24 03:11:04 - Extract solid (k+1)-mers for k = 21 
2023-02-24 03:19:46 - Build graph for k = 21 
2023-02-24 03:22:47 - Assemble contigs fr

In [ ]:
# consider deleting the bams after we get the qualimap reports - we don't need them anymore